# Data Carpentry on the Aggrigated Zip Code Data

My goal is to make a single dataframe per category containing all ten years worth of data for that category. The categories are as follows.

* Home Owners - loss;       homelos
* Home Owners - exposure;   homeexp
* Farm Owners - loss;       farmlos
* Farm Owners - exposure;   farmexp
* Mobile Home - loss;       mobilelos
* Mobile Home - exposure;   mobileexp
* Earthquake - loss;        earthlos
* Earthquake - exposure;    earthexp
* Automobile - loss;        autolos
* Automobile - exposure;    autoexp

#### Import necessary packages.

In [1]:
import os
import pandas as pd
import sqlalchemy as sqla
import sqlite3 as sql3

#### Define a function.

The first function walks through a designated folder containing data files. These files are stored in folders by year and caregory. It then loads the files into separate dataframes, and then concatenates the different years of data together into a single dataframe of all years for that category. Lastly, it loads the data into a sqlite database.

In [2]:
def walk_and_process(directory):
    '''
    Function to walk through a directory and process all data files into pandas dataframes.

    Parameters:
    directory (str): The directory to walk through.
    '''
    
    # Create an empty list to store the directories
    DIRS = []

    # Walk through the directory and store the directories in the DIRS list
    for root, dirs, files in os.walk(directory):

        for file in files:

            if file.startswith('.'):

                pass

            else:

                file_path = os.path.join(root, file)
                
                # Split the file path and store the directories in the DIRS list
                lst = file_path.split('/')

                # Append the year, category, and filename to the DIRS list
                DIRS.append(lst[8:])

    # Create a dataframe for each file in the directory
    for root, dirs, files in os.walk(directory):

        for file in files:

            globals()[file] = pd.read_table(os.path.join(root, file), sep='\t')

    # Create a master dataframe for each type of data
    master_dfs = [
        'homelos',
        'homeexp',
        'farmlos',
        'farmexp',
        'mobilelos',
        'mobileexp',
        'earthlos',
        'earthexp',
        'autolos',
        'autoexp'    
    ]

    # Process the data in each dataframe
    for df in master_dfs:

        globals()[df] = pd.DataFrame()

        for d in DIRS:

            # Add the year to the dataframe.
            globals()[d[-1]]['year'] = d[0]

            for c in globals()[d[-1]].columns:
                
                # Check if the column is an object
                if globals()[d[-1]][c].dtype == 'object':
                    
                    # Remove $ and , from the columns
                    globals()[d[-1]][c] = globals()[d[-1]][c].str.replace('$', '')
                    globals()[d[-1]][c] = globals()[d[-1]][c].str.replace(',', '')
                
                try:

                    # Convert the column to numeric
                    globals()[d[-1]][c] = pd.to_numeric(globals()[d[-1]][c], errors='coerce')

                except Exception as e:

                    print(e)
                    #print(f'Could not convert column {c} to numeric in {df[-1]}')

            if df in d[-1]:

                # Concatenate the dataframes that are the same category
                globals()[df] = pd.concat([globals()[df], globals()[d[-1]]], ignore_index=True)

    # Drop columns that are all NaN
    for df in master_dfs:
        
        globals()[df].dropna(axis=1, how='all', inplace=True)
        globals()[df]['zip'] = globals()[df]['zip'].astype(str)
        globals()[df]['year'] = globals()[df]['year'].astype(str)

    # return the list of dataframe names
    return master_dfs



#### Define the directory and run the function.

In [3]:
directory_to_walk = '/Users/skylar_calvin/Library/CloudStorage/OneDrive-UniversityofMissouri/Data/All Data Requested'

dfs = walk_and_process(directory_to_walk)

#### Connect to the SQLite database created in previous notebooks.

In [4]:
engine = sqla.create_engine('sqlite:///life_data.db')

#### Now load the data into the life_data sqlite database.

In [5]:
for df in dfs:
    
    globals()[df].to_sql(name = df, con = engine, if_exists = 'replace', index = False)
    